## AUTOGLUON experiments using the formatted CSV files as inputs

### check the environment first 

In [1]:
import sys

In [2]:
print(sys.executable) 

/home/nicolasf/anaconda3/envs/ML/bin/python


In [3]:
# Parameters 

GCM = 'ECMWF'
var_name = 'TMEAN'
target_type = 'cat3_categories'
region_name = 'NNI'
skpca = True 
standardized = False

### load external modules 

In [4]:
%matplotlib inline

In [5]:
import os
import pathlib
from shutil import copytree, rmtree

In [6]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [7]:
HOME = pathlib.Path.home()

In [8]:
from matplotlib import pyplot as plt

In [9]:
import proplot as plot

In [10]:
import numpy as np
import pandas as pd

In [11]:
import sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import pca

### import autogluon, tabular prediction, see [https://autogluon.mxnet.io/tutorials/tabular_prediction/tabular-quickstart.html](https://autogluon.mxnet.io/tutorials/tabular_prediction/tabular-quickstart.html)

In [12]:
import autogluon as ag
from autogluon import TabularPrediction as task

/home/nicolasf/anaconda3/envs/ML/lib/python3.7/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


### load local modules 

In [13]:
sys.path.append('../../../../ml4seas/')

In [14]:
from GCM import prepare_data_CSV_to_AUTOML

In [15]:
dpath = HOME / 'research' / 'Smart_Ideas' / 'outputs' / 'CSVs'

In [16]:
list(dpath.glob("*.csv"))

[PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_RAIN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_TMEAN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_TMEAN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_RAIN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_RAIN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_TMEAN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_TMEAN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_RAIN_training_set.csv')]

In [17]:
if standardized: 
    train_data = pd.read_csv(dpath / f'GCMs_std_and_targets_cat3_and_anomalies_{var_name}_training_set.csv', index_col=0, parse_dates=True) 
    test_data = pd.read_csv(dpath / f'GCMs_std_and_targets_cat3_and_anomalies_{var_name}_test_set.csv', index_col=0, parse_dates=True)
else: 
    train_data = pd.read_csv(dpath / f'GCMs_and_targets_cat3_and_anomalies_{var_name}_training_set.csv', index_col=0, parse_dates=True) 
    test_data = pd.read_csv(dpath / f'GCMs_and_targets_cat3_and_anomalies_{var_name}_test_set.csv', index_col=0, parse_dates=True)    

### get the training data 

In [18]:
train_data, GCMs_name_train, _, _ = prepare_data_CSV_to_AUTOML(train_data, GCM=GCM, region_name=region_name, target_type=target_type, scaling=False, doPCA=False)

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [19]:
train_data.shape

(288, 4930)

In [20]:
train_data.columns

Index(['(-70.0, 70.0)', '(-70.0, 72.5)', '(-70.0, 75.0)', '(-70.0, 77.5)',
       '(-70.0, 80.0)', '(-70.0, 82.5)', '(-70.0, 85.0)', '(-70.0, 87.5)',
       '(-70.0, 90.0)', '(-70.0, 92.5)',
       ...
       '(60.0, 280.0)', '(60.0, 282.5)', '(60.0, 285.0)', '(60.0, 287.5)',
       '(60.0, 290.0)', '(60.0, 292.5)', '(60.0, 295.0)', '(60.0, 297.5)',
       '(60.0, 300.0)', 'NNI_cat3_categories'],
      dtype='object', length=4930)

### get the test data  

In [21]:
test_data, GCMs_name_test, _, _ = prepare_data_CSV_to_AUTOML(test_data, GCM=GCM, region_name=region_name, target_type=target_type, scaling=False, doPCA=False)

In [22]:
test_data.shape

(33, 4930)

### stratified k-fold 

In [23]:
from sklearn.model_selection import StratifiedKFold

### the percentage of variance to keep 

In [24]:
percent_variance = 0.9

### root path for saving the parameters of all the AUTOGLUON experiments 

In [25]:
saved_models = pathlib.Path('./saved_models/AUTOGLUON_v2/')

In [26]:
opath = saved_models.joinpath(f'./autogluon_exp_SKPCA_{GCM}_pred_{region_name}_reg_{var_name}_targetvar_{target_type}_target_type')

In [27]:
if not opath.exists(): 
    opath.mkdir(parents=True)

### checks on the shape and content of the training and test data 

In [28]:
train_data.shape

(288, 4930)

In [29]:
train_data.columns

Index(['(-70.0, 70.0)', '(-70.0, 72.5)', '(-70.0, 75.0)', '(-70.0, 77.5)',
       '(-70.0, 80.0)', '(-70.0, 82.5)', '(-70.0, 85.0)', '(-70.0, 87.5)',
       '(-70.0, 90.0)', '(-70.0, 92.5)',
       ...
       '(60.0, 280.0)', '(60.0, 282.5)', '(60.0, 285.0)', '(60.0, 287.5)',
       '(60.0, 290.0)', '(60.0, 292.5)', '(60.0, 295.0)', '(60.0, 297.5)',
       '(60.0, 300.0)', 'NNI_cat3_categories'],
      dtype='object', length=4930)

In [30]:
target_col = f"{region_name}_{target_type}"; print(target_col)

NNI_cat3_categories


### initialise a stratified K-Fold object, which will return train and test indices 

In [31]:
kfold = StratifiedKFold(n_splits=10).split(train_data.drop(labels=[target_col],axis=1).values, train_data.loc[:,target_col].values)

In [32]:
test_indices = []
# y_preds = []
# leader_board = []
# perfs = []

# for k, (train, test) in enumerate(kfold):
    
#     X_train = train_data.drop(labels=[target_col],axis=1).iloc[train,:]
    
#     y_train = train_data.loc[:,target_col].iloc[train,]
    
#     X_test = train_data.drop(labels=[target_col],axis=1).iloc[test,:]
    
#     y_test = X_df.loc[:,target_col].iloc[test,]
    
#     df_train = X_train.copy() 
#     df_train.loc[:, target_col] = y_train
    
#     df_test = X_test.copy() 
#     df_test.loc[:,target_col] = y_test
    
    
#      # fit the task predictor on the training set DataFrame 
#     predictor = task.fit(train_data=df_train, label=target_col, auto_stack=True, output_directory=opath)
    
#     # predict the probabilities for each class from the test set features DataFrame (droping the target values column)
#     # y_pred_proba = predictor.predict_proba(df_test.drop(labels=[region_name],axis=1))
    
#     # predict the class value itself
#     y_pred = predictor.predict(df_test.drop(labels=[target_col],axis=1))
    
#     # records the probabilities for the classes on the test set 
#     y_preds.append(y_pred)
    
#     # get the leaderboard DataFrame 
#     d = predictor.leaderboard(silent=True)
    
#     # records the leaderboard DataFrame 
#     leader_board.append(d)
    
#     perfs.append(predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True, silent=True))
    
#     print(f"EXITING FOLD {k} ---- ")   

In [33]:
test_indices = []
y_preds = []
leader_board = []
perfs = []

for k, (train, test) in enumerate(kfold):
    
    print(f"ENTERING FOLD {k} ---- ")
    
    # saves the test indices
    test_indices.append(test)
    
    # get the numpy array containing the training set initial features (grid points)
    X_train = train_data.drop(labels=[target_col],axis=1).values[train]
    
    # get the numpy array containing the training set target values (y)
    y_train =  train_data.loc[:,target_col].values[train]
    
    # get the numpy array containing the test set initial features (grid points)
    X_test = train_data.drop(labels=[target_col],axis=1).values[test]
    
    # get the numpy array containing the test set target values (y)
    y_test = train_data.loc[:,target_col].values[test]
    
    # -----------
    # standardize 
    
    # initialise the scaler (standard scaler)
    scaler = StandardScaler() 
    
    # fit on the training set features array, and transform to obtain standardized values
    X_train_std = scaler.fit_transform(X_train)
    
    # apply the transformation on the test set initial features 
    X_test_std = scaler.transform(X_test)
    
    # -----------------------------
    # Principal Component Analysis 
    
    # instantiate the pca class, with percent of variance to keep  
    
    skpca = pca.PCA(n_components=percent_variance)
    
    # fit on the training initial (standardized) fedatures array, and transform to obtain the PCs
    X_train_PC = skpca.fit_transform(X_train_std)
    
    # apply the transformation on the test set standardized features 
    X_test_PC = skpca.transform(X_test_std)
    
    # assign the training set PCs to a DataFrame 
    df_train = pd.DataFrame(X_train_PC) 
    
    # add the target values to the training DataFrame 
    df_train.loc[:,target_col] = y_train
    
    # assign the test set Pcs to a DataFrame 
    df_test = pd.DataFrame(X_test_PC) 
    
    # add the target values to the test DataFrame
    df_test.loc[:,target_col] = y_test
    
    # fit the task predictor on the training set DataFrame 
    predictor = task.fit(train_data=df_train, label=target_col, auto_stack=True, output_directory=opath)
    
    # predict the probabilities for each class from the test set features DataFrame (droping the target values column)
#     y_pred_proba = predictor.predict_proba(df_test.drop(labels=[region_name],axis=1))
    
    # predict the class value itself
    y_pred = predictor.predict(df_test.drop(labels=[target_col],axis=1))
    
    # records the probabilities for the classes on the test set 
    y_preds.append(y_pred)
    
    # get the leaderboard DataFrame 
    d = predictor.leaderboard(silent=True)
    
    # records the leaderboard DataFrame 
    leader_board.append(d)
    
    perfs.append(predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True, silent=True))
    
    print(f"EXITING FOLD {k} ---- ")

Beginning AutoGluon training ...
AutoGluon will save models to saved_models/AUTOGLUON_v2/autogluon_exp_SKPCA_ECMWF_pred_NNI_reg_TMEAN_targetvar_cat3_categories_target_type/
Train Data Rows:    258
Train Data Columns: 35
Preprocessing data ...
Here are the first 10 unique label values in your data:  [1 2 3]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed)
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Feature Generator processed 258 data points with 34 features
Original Features:
	float features: 34
Generated Features:
	int features: 0
All Features:
	float features: 34
	int features: 0
	Data preprocessing and feature engineering runtime = 0.08s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will

ENTERING FOLD 0 ---- 


/home/nicolasf/anaconda3/envs/ML/lib/python3.7/imp.py:342: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return _load(spec)
Fitting model: RandomForestClassifierGini_STACKER_l0 ...
	0.6085	 = Validation accuracy score
	2.63s	 = Training runtime
	0.76s	 = Validation runtime
Fitting model: RandomForestClassifierEntr_STACKER_l0 ...
	0.6202	 = Validation accuracy score
	2.67s	 = Training runtime
	0.65s	 = Validation runtime
Fitting model: ExtraTreesClassifierGini_STACKER_l0 ...
	0.6124	 = Validation accuracy score
	2.13s	 = Training runtime
	0.7s	 = Validation runtime
Fitting model: ExtraTreesClassifierEntr_STACKER_l0 ...
	0.6279	 = Validation accuracy score
	2.12s	 = Training runtime
	0.55s	 = Validation runtime
Fitting model: KNeighborsClassifierUnif_STACKER_l0 ...
	0.6434	 = Validation accuracy score
	0.18s	 = Training runtime
	0.58s	 = Validation runtime
Fitting model

EXITING FOLD 0 ---- 
ENTERING FOLD 1 ---- 


Here are the first 10 unique label values in your data:  [1 2 3]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed)
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Feature Generator processed 258 data points with 32 features
Original Features:
	float features: 32
Generated Features:
	int features: 0
All Features:
	float features: 32
	int features: 0
	Data preprocessing and feature engineering runtime = 0.13s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Fitting model: RandomForestClassifierGini_STACKER_l0 ...
	0.6047	 = Validation accuracy score
	2.67s	 = Training runtime
	0.87s	 = Validation runtime
Fitting model: RandomForestClassifierEnt

...

	0.6269	 = Validation accuracy score
	2.56s	 = Training runtime
	0.71s	 = Validation runtime
Fitting model: RandomForestClassifierEntr_STACKER_l0 ...
	0.6192	 = Validation accuracy score
	2.59s	 = Training runtime
	0.62s	 = Validation runtime
Fitting model: ExtraTreesClassifierGini_STACKER_l0 ...
	0.6154	 = Validation accuracy score
	2.14s	 = Training runtime
	0.57s	 = Validation runtime
Fitting model: ExtraTreesClassifierEntr_STACKER_l0 ...
	0.6192	 = Validation accuracy score
	2.13s	 = Training runtime
	0.55s	 = Validation runtime
Fitting model: KNeighborsClassifierUnif_STACKER_l0 ...
	0.6115	 = Validation accuracy score
	0.15s	 = Training runtime
	0.58s	 = Validation runtime
Fitting model: KNeighborsClassifierDist_STACKER_l0 ...
	0.6192	 = Validation accuracy score
	0.19s	 = Training runtime
	0.58s	 = Validation runtime
Fitting model: LightGBMClassifier_STACKER_l0 ...
	0.6385	 = Validation accuracy score
	1.51s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: Catboost

EXITING FOLD 8 ---- 
ENTERING FOLD 9 ---- 


	0.5824	 = Validation accuracy score
	2.65s	 = Training runtime
	0.65s	 = Validation runtime
Fitting model: RandomForestClassifierEntr_STACKER_l0 ...
	0.5556	 = Validation accuracy score
	2.65s	 = Training runtime
	0.72s	 = Validation runtime
Fitting model: ExtraTreesClassifierGini_STACKER_l0 ...
	0.6169	 = Validation accuracy score
	2.15s	 = Training runtime
	0.55s	 = Validation runtime
Fitting model: ExtraTreesClassifierEntr_STACKER_l0 ...
	0.5939	 = Validation accuracy score
	2.14s	 = Training runtime
	0.57s	 = Validation runtime
Fitting model: KNeighborsClassifierUnif_STACKER_l0 ...
	0.5977	 = Validation accuracy score
	0.16s	 = Training runtime
	0.58s	 = Validation runtime
Fitting model: KNeighborsClassifierDist_STACKER_l0 ...
	0.6169	 = Validation accuracy score
	0.17s	 = Training runtime
	0.58s	 = Validation runtime
Fitting model: LightGBMClassifier_STACKER_l0 ...
	0.6054	 = Validation accuracy score
	1.75s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: Catboost

EXITING FOLD 9 ---- 


/home/nicolasf/anaconda3/envs/ML/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
len(leader_board)

10

In [35]:
top = []
for i in range(len(leader_board)): 
    top.append(leader_board[i].iloc[[0],:])

In [36]:
top = pd.concat(top, axis=0)

In [37]:
top

,model,score_val,fit_time,pred_time_val,stack_level
10,weighted_ensemble_k0_l1,0.689922,0.383616,0.000739,1
10,weighted_ensemble_k0_l1,0.666667,0.384336,0.000791,1
10,weighted_ensemble_k0_l1,0.694981,0.382756,0.000773,1
10,weighted_ensemble_k0_l1,0.706564,0.367373,0.000731,1
10,weighted_ensemble_k0_l1,0.660232,0.374954,0.000756,1
10,weighted_ensemble_k0_l1,0.671815,0.357203,0.000745,1
10,weighted_ensemble_k0_l1,0.625483,0.355526,0.000756,1
10,weighted_ensemble_k0_l1,0.661538,0.344448,0.000738,1
10,weighted_ensemble_k0_l1,0.653846,0.331419,0.000798,1
10,weighted_ensemble_k0_l1,0.643678,0.325032,0.000819,1
